In [3]:
ls

chroma_db_eeve/  eeve_Rag.ipynb  medicines_info.txt


In [3]:
import os

# 3번 GPU만 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

/usr/anaconda3/envs/med/lib/python3.12/importlib/__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [8]:
# 1. HuggingFace 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

# 2. Chroma DB 설정 (DB를 저장할 경로)
persist_directory = "chroma_db_eeve"  # 저장 경로 설정
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# 3. 텍스트 파일 읽기 및 데이터 분할
with open('medicines_info.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# '=====' 구분자로 나누기
chunks = text.split('=====')
print(f"분할된 문서 수: {len(chunks)}")

분할된 문서 수: 7037


In [8]:
print(chunks[0])

약품 품목 기준 코드 : '201507217'
제품명 : '1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)'
제품영문명 : 'Povidine Throat Spray Sol.'
주성분 : '포비돈 요오드'
주성분영문 : 'Povidone Iodine'
첨가제 : '프로필렌글리콜,청포도71334,자일리톨,유칼립톨,효소처리스테비아,L-멘톨,요오드화칼륨,바닐라향 HF-63374,에탄올,농글리세린,정제수'
의약품 분류 기준 : '일반의약품'



In [9]:
# 4. Chroma DB에 데이터 저장
documents = []
metadatas = []

# 각 chunk에 대해 문서와 메타데이터를 생성
for idx, chunk in enumerate(chunks):
    documents.append(chunk)
    metadatas.append({"chunk_id": idx})  # 각 chunk에 대한 메타데이터 (필요 시 추가 정보 저장 가능)

# 벡터 스토어에 문서 및 임베딩 저장
vector_store.add_texts(documents, metadatas=metadatas)

# 5. Chroma DB 저장 완료
vector_store.persist()
print("Chroma DB 저장 완료.")

KeyboardInterrupt: 

In [9]:
# 2. Chroma DB 로드 (임베딩 함수와 함께 설정)
persist_directory = "/home/students/cs/sjuhun1/EEVE_rag/chroma_db_eeve"  # 이전에 저장한 DB의 경로
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [18]:
# 3. 질문 입력
user_message = "what is 나잘스프레이오리지날?"

In [19]:
# 4. Chroma DB에서 유사한 문서 검색
retrieved_docs = vector_store.similarity_search(user_message)

In [20]:
# 검색된 문서들을 하나의 텍스트로 병합
context = "\n".join([doc.page_content for doc in retrieved_docs])

print(context)


약품 품목 기준 코드 : '201404778'
제품명 : '1.나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Original) 2.나잘스프레이엑스트라모이스쳐라이징(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Extra Moisturizing)'
제품영문명 : 'Nasal Spray Original,  Nasal Spray Extra Moisturizing'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '벤잘코늄염화물,에데트산나트륨수화물,정제수,농글리세린'
의약품 분류 기준 : '일반의약품'


약품 품목 기준 코드 : '201210198'
제품명 : '퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant,

VM PLUS Nasal Spray, Family Care Nasal Spray Original,

Family Care Severe Congestion Nasal Spray ,

Leader Extra Moisturizing Nasal Spray Original,

Leader Severe Congestion Nasal Spray)'
제품영문명 : 'Pure Aid Nasal Spray Original'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '인산이수소나트륨수화물,벤질알코올,프로필렌글리콜,dl-캄파,폴리소르베이트80,인산수소나트륨수화물,폴리에틸렌글리콜,에데트산나트륨수화물,포비돈,벤잘코늄염화물,유칼립톨,L-멘톨,정제수'
의약품 분류 기준 : '일반의약품'


약품 품

In [1]:
import transformers
import torch

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [22]:
PROMPT = '''
다음 문서의 내용을 바탕으로 질문에 답하세요:\n
{}
\n
'''.format(context)

print(PROMPT)


다음 문서의 내용을 바탕으로 질문에 답하세요:


약품 품목 기준 코드 : '201404778'
제품명 : '1.나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Original) 2.나잘스프레이엑스트라모이스쳐라이징(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Extra Moisturizing)'
제품영문명 : 'Nasal Spray Original,  Nasal Spray Extra Moisturizing'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '벤잘코늄염화물,에데트산나트륨수화물,정제수,농글리세린'
의약품 분류 기준 : '일반의약품'


약품 품목 기준 코드 : '201210198'
제품명 : '퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant,

VM PLUS Nasal Spray, Family Care Nasal Spray Original,

Family Care Severe Congestion Nasal Spray ,

Leader Extra Moisturizing Nasal Spray Original,

Leader Severe Congestion Nasal Spray)'
제품영문명 : 'Pure Aid Nasal Spray Original'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '인산이수소나트륨수화물,벤질알코올,프로필렌글리콜,dl-캄파,폴리소르베이트80,인산수소나트륨수화물,폴리에틸렌글리콜,에데트산나트륨수화물,포비돈,벤잘코늄염화물,유칼립톨,L-멘톨,정제수

In [23]:
instruction = user_message

print(instruction)

what is 나잘스프레이오리지날?


In [24]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"][len(prompt):])

나잘스프레이오리지날은 옥시메타졸린염산염을 주성분으로 하는 일반의약품입니다. 옥시메타졸린염산염은 인두비염, 천식, 비염 등의 증상을 완화시키기 위해 사용되는 약물입니다. '나잘스프레이오리지날'은 이를 목적으로 하는 인두 스프레이 형태의 제품입니다. '오리지널'이라는 이름은 이 제품이 기본적인 형태로, 추가적인 보습 성분이 포함되지 않은 것을 의미할 수 있습니다. '나잘스프레이엑스트라모이스쳐라이징'은 이와 달리 추가적인 보습 성분이 포함된 형태로, 더 건조한 인두를 보다 잘 관리하기 위해 설계된 제품입니다.
